# Import Libraries

In [139]:
import pandas as pd
import datetime

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge, Lasso 
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import r2_score

# EDA

In [2]:
# importing the data

df = pd.read_csv("Walmart_Store_sales.csv")

In [3]:
# How does the data look like
df.head(20)

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092
5,4.0,28-05-2010,1857533.70,0.0,NaN,2.756,126.160226,7.896
6,15.0,03-06-2011,695396.19,0.0,69.80,4.069,134.855161,7.658
7,20.0,03-02-2012,2203523.20,0.0,39.93,3.617,213.023622,6.961
8,14.0,10-12-2010,2600519.26,0.0,30.54,3.109,NaN,NaN
9,3.0,NaN,418925.47,0.0,60.12,3.555,224.132020,6.833


In [4]:
df.describe(include='all')

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000


We can observe from the talble above that there are missing values due to the difference in counts, and there seem to be abnormal temperature values. Let's explore more 

In [5]:
df.columns

Index(['Store', 'Date', 'Weekly_Sales', 'Holiday_Flag', 'Temperature',
       'Fuel_Price', 'CPI', 'Unemployment'],
      dtype='object')

In [6]:
df.dtypes

Store           float64
Date             object
Weekly_Sales    float64
Holiday_Flag    float64
Temperature     float64
Fuel_Price      float64
CPI             float64
Unemployment    float64
dtype: object

In [7]:
# exploring the values of the store column

df['Store'].value_counts()

Store
3.0     15
1.0     11
18.0    10
19.0     9
5.0      9
14.0     9
13.0     9
7.0      8
17.0     8
2.0      8
8.0      8
6.0      7
20.0     7
4.0      7
12.0     5
10.0     5
15.0     4
16.0     4
9.0      4
11.0     3
Name: count, dtype: int64

we can observe that not all branches are reported evenly with the same frequency, like for instance branch no. 11 was has only three values of weekly sales where as store(3) 15 times 

In [136]:
# number of stores

print(f" there are {len(df['Store'].unique())} stores ")

 there are 20 stores 


In [9]:
# examening the dates
df['Date'].sort_values().unique()

array(['01-06-2012', '01-10-2010', '02-03-2012', '02-04-2010',
       '02-07-2010', '03-02-2012', '03-06-2011', '03-09-2010',
       '04-05-2012', '04-06-2010', '05-02-2010', '05-08-2011',
       '06-01-2012', '06-05-2011', '06-07-2012', '06-08-2010',
       '07-01-2011', '07-09-2012', '07-10-2011', '08-07-2011',
       '09-07-2010', '09-12-2011', '10-02-2012', '10-06-2011',
       '10-09-2010', '10-12-2010', '11-05-2012', '11-06-2010',
       '11-11-2011', '12-02-2010', '12-03-2010', '12-08-2011',
       '12-10-2012', '12-11-2010', '13-01-2012', '13-04-2012',
       '13-05-2011', '14-05-2010', '14-09-2012', '15-04-2011',
       '15-07-2011', '15-10-2010', '16-03-2012', '16-04-2010',
       '16-12-2011', '17-02-2012', '17-09-2010', '17-12-2010',
       '18-02-2011', '18-05-2012', '18-06-2010', '18-11-2011',
       '19-02-2010', '19-08-2011', '19-10-2012', '19-11-2010',
       '20-01-2012', '20-04-2012', '20-05-2011', '20-08-2010',
       '22-04-2011', '22-06-2012', '22-07-2011', '23-09

In [10]:
df['CPI']

0      214.777523
1      128.616064
2      130.719581
3      214.556497
4      212.412888
          ...    
145    182.442420
146           NaN
147    126.111903
148    219.007525
149    137.923067
Name: CPI, Length: 150, dtype: float64

In [11]:
# transofrming the date column from string to date
df['Date'] = pd.to_datetime(df['Date'])

# we will receive a warning of the format of the date since we did not choose one, no worries

C:\Users\Lionel\AppData\Local\Temp\ipykernel_23656\2394721818.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Date'] = pd.to_datetime(df['Date'])


In [137]:
df['Date'].sort_values().unique()

<DatetimeArray>
['2010-02-05 00:00:00', '2010-02-12 00:00:00', '2010-02-19 00:00:00',
 '2010-02-26 00:00:00', '2010-03-12 00:00:00', '2010-03-26 00:00:00',
 '2010-04-02 00:00:00', '2010-04-16 00:00:00', '2010-04-30 00:00:00',
 '2010-05-14 00:00:00', '2010-05-28 00:00:00', '2010-06-04 00:00:00',
 '2010-06-11 00:00:00', '2010-06-18 00:00:00', '2010-06-25 00:00:00',
 '2010-07-02 00:00:00', '2010-07-09 00:00:00', '2010-07-30 00:00:00',
 '2010-08-06 00:00:00', '2010-08-20 00:00:00', '2010-08-27 00:00:00',
 '2010-09-10 00:00:00', '2010-09-17 00:00:00', '2010-09-24 00:00:00',
 '2010-10-01 00:00:00', '2010-10-15 00:00:00', '2010-11-12 00:00:00',
 '2010-11-19 00:00:00', '2010-11-26 00:00:00', '2010-12-10 00:00:00',
 '2010-12-17 00:00:00', '2010-12-31 00:00:00', '2011-01-07 00:00:00',
 '2011-02-18 00:00:00', '2011-03-25 00:00:00', '2011-04-15 00:00:00',
 '2011-04-22 00:00:00', '2011-05-06 00:00:00', '2011-05-13 00:00:00',
 '2011-05-20 00:00:00', '2011-05-27 00:00:00', '2011-06-03 00:00:00',
 '20

observation : we would expect to have 4 weekly reports in a month. Unfortunatley this is not always the case in our dataset, like in 2011 we had only one report in January i.e on record 

In [12]:
df.dtypes

Store                  float64
Date            datetime64[ns]
Weekly_Sales           float64
Holiday_Flag           float64
Temperature            float64
Fuel_Price             float64
CPI                    float64
Unemployment           float64
dtype: object

# B) Preprocessing

Start by checking the percentage of empty fields in the data set

In [162]:
df.isnull().sum()/df.shape[0]*100

Store            0.000000
Date            12.000000
Weekly_Sales     9.333333
Holiday_Flag     8.000000
Temperature     12.000000
Fuel_Price       9.333333
CPI              8.000000
Unemployment    10.000000
dtype: float64

we can notice the following:
1. There's a 9% of empty targets 'weekly sales', these rows will be deleted as it would be meaningless not to have a target value.How will the model do its work?
2. The temperature , the CPI and the unemplyment rate missing values could be filled with the average "mean value" or median while preprocessing.
3. As for the "holiday" flag 9% is a considerable amount however, if we discard them this might affect the data set especially that it has a limited number of rows consequnetly we will replace it with the most frequent occurence
4. As for the "Date" column, 12% is really a huge number consequently it is preffered to be dropped, or let's check the percentage after deleting the empty targets

### B.1) Removing empty targets 
It would be meaningless to have empty targets, so we will delete the rows that don't have target results

In [138]:
print(f'The current no of rows is {df.shape[0]}')
print(f'The number of rows with empty targets is {df["Weekly_Sales"].isnull().sum()} ')


The current no of rows is 136
The number of rows with empty targets is 0 


In [14]:
mask = df['Weekly_Sales'].notna()
df = df.loc[mask,:]
print(f"data set reduced to {df.shape[0]}")

data set reduced to 136


### B.2) Preprocessing the features

In [15]:
# Let's check the new status
df.isnull().sum()/df.shape[0]*100

Store            0.000000
Date            13.235294
Weekly_Sales     0.000000
Holiday_Flag     8.088235
Temperature     11.029412
Fuel_Price       8.823529
CPI              8.088235
Unemployment    10.294118
dtype: float64

In my opinion 13% of missing values is really too much, on the other hand we cannot simply implement the impute because it is a date. Consequently it would be wise to drop it from datascience point of view because we have a small dataset. 
Although i personaly believe that it plays a role in estimating the weekly sales

In [16]:
df.describe()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,136.000000,118,1.360000e+02,125.000000,121.000000,124.000000,125.000000,122.000000
mean,10.014706,2011-04-23 04:28:28.474576384,1.249536e+06,0.072000,60.853967,3.316992,178.091144,7.665582
min,1.000000,2010-02-05 00:00:00,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,2010-07-30 00:00:00,6.050757e+05,0.000000,45.220000,2.838500,131.637000,6.690000
50%,10.000000,2011-04-29 00:00:00,1.261424e+06,0.000000,62.250000,3.451000,196.919506,7.477000
75%,15.250000,2011-12-31 18:00:00,1.806386e+06,0.000000,75.950000,3.724000,214.878556,8.150000
max,20.000000,2012-10-19 00:00:00,2.771397e+06,1.000000,91.650000,4.193000,226.968844,14.313000
std,6.124614,NaN,6.474630e+05,0.259528,18.514432,0.479540,40.243105,1.619428


### B.3) removing the outliers

**IMPORTANT NOTE(1)**
if we implement our masks directly to filter the outliers we will find out that all the rows containing 'na' would be filtered out for each column , causing a huge reduction in the number of rows for the dataset.

Consequently it would be wise if we replace the 'na' with the mean of each column as if we are playing the role of the imputer in advance. But wait.... this might chnage the values of the 'mean' and the 'std' too which we ocunt on them to exclude the outliers

this is only for the columns on which we will implement the outliers filtering which are Temperature, Fuel_Price, CPI and unemployment except the holidays

**IMPORTANT NOTE(2)** each time we implement a mask on a column we will result in a differnet number of rows which affects the mean and 'std' values of the other columns. This calulation falsifies the real range of the outilers to be excluded.

**__What to do?__**
**The solution** is that we will make a new dataset a copy of the old one and place all the results in the new one. whereas the calculations of the mean and std ar based on the original datat set 😎

In [17]:
# first we will decalre our new dataframe on which we will implement our machine learning
df2 = df.copy()

Why did we do this?
Because if we mask directly the outliers all 'na' rows will be deleted
on the other hand if i replce the 'na' with the mean, the range of outliers will be affected as a result

consequently we will calculate the mean and std from the original dataframe (df) and use these values to fill in the na in the new dataset and exclude the outliers in the NEW dataset 

In [18]:
list_cols = ['Temperature', 'Fuel_Price', 'CPI' ,'Unemployment']

for feature in list_cols:
    
    # calculate the mean based on the original dataframe
    mean_value = df[feature].mean()         
    
    #filling the 'na' in new dataset with the mean values 
    df2[feature] = df2[feature].fillna(mean_value) 
    
    
    print(f"The mean value for the {feature} is {mean_value}")
    print('Completed filling the na values in the new dataframe with mean value')
    print()




The mean value for the Temperature is 60.85396694214876
Completed filling the na values in the new dataframe with mean value

The mean value for the Fuel_Price is 3.316991935483871
Completed filling the na values in the new dataframe with mean value

The mean value for the CPI is 178.09114404399998
Completed filling the na values in the new dataframe with mean value

The mean value for the Unemployment is 7.665581967213115
Completed filling the na values in the new dataframe with mean value



In [20]:
df2.head(20)

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,2011-02-18,1572117.54,NaN,59.610000,3.045000,214.777523,6.858000
1,13.0,2011-03-25,1807545.43,0.0,42.380000,3.435000,128.616064,7.470000
3,11.0,NaT,1244390.03,0.0,84.570000,3.316992,214.556497,7.346000
4,6.0,2010-05-28,1644470.66,0.0,78.890000,2.759000,212.412888,7.092000
5,4.0,2010-05-28,1857533.70,0.0,60.853967,2.756000,126.160226,7.896000
6,15.0,2011-06-03,695396.19,0.0,69.800000,4.069000,134.855161,7.658000
7,20.0,2012-02-03,2203523.20,0.0,39.930000,3.617000,213.023622,6.961000
8,14.0,2010-12-10,2600519.26,0.0,30.540000,3.109000,178.091144,7.665582
9,3.0,NaT,418925.47,0.0,60.120000,3.555000,224.132020,6.833000
10,8.0,2011-08-19,895066.50,0.0,82.920000,3.554000,219.070197,6.425000


In [19]:
# how many na do we have in our new dataframe
df2.isnull().sum()/df2.shape[0]*100

Store            0.000000
Date            13.235294
Weekly_Sales     0.000000
Holiday_Flag     8.088235
Temperature      0.000000
Fuel_Price       0.000000
CPI              0.000000
Unemployment     0.000000
dtype: float64

In [173]:
# whereas nothing changed in the old dataframe
df.isnull().sum()/df.shape[0]*100

Store            0.000000
Date            13.235294
Weekly_Sales     0.000000
Holiday_Flag     8.088235
Temperature     11.029412
Fuel_Price       8.823529
CPI              8.088235
Unemployment    10.294118
dtype: float64

Wonderful, now we will filter the outliers based on the range.
As for the Holiday flag we will leave it for the impute to calucalte the missing

In [174]:
df2.describe()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,136.000000,118,1.360000e+02,125.000000,136.000000,136.000000,136.000000,136.000000
mean,10.014706,2011-04-23 04:28:28.474576384,1.249536e+06,0.072000,60.853967,3.316992,178.091144,7.665582
min,1.000000,2010-02-05 00:00:00,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,2010-07-30 00:00:00,6.050757e+05,0.000000,48.072500,2.853750,132.464884,6.833000
50%,10.000000,2011-04-29 00:00:00,1.261424e+06,0.000000,60.853967,3.316992,189.613975,7.661791
75%,15.250000,2011-12-31 18:00:00,1.806386e+06,0.000000,75.175000,3.689750,214.756314,8.100750
max,20.000000,2012-10-19 00:00:00,2.771397e+06,1.000000,91.650000,4.193000,226.968844,14.313000
std,6.124614,NaN,6.474630e+05,0.259528,17.455574,0.457731,38.568739,1.533160


In [175]:
df.describe()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,136.000000,118,1.360000e+02,125.000000,121.000000,124.000000,125.000000,122.000000
mean,10.014706,2011-04-23 04:28:28.474576384,1.249536e+06,0.072000,60.853967,3.316992,178.091144,7.665582
min,1.000000,2010-02-05 00:00:00,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,2010-07-30 00:00:00,6.050757e+05,0.000000,45.220000,2.838500,131.637000,6.690000
50%,10.000000,2011-04-29 00:00:00,1.261424e+06,0.000000,62.250000,3.451000,196.919506,7.477000
75%,15.250000,2011-12-31 18:00:00,1.806386e+06,0.000000,75.950000,3.724000,214.878556,8.150000
max,20.000000,2012-10-19 00:00:00,2.771397e+06,1.000000,91.650000,4.193000,226.968844,14.313000
std,6.124614,NaN,6.474630e+05,0.259528,18.514432,0.479540,40.243105,1.619428


In [21]:
# Excluding the outliers
# for the following columns "features" : Temperature, Fuel_Price, CPI and unemployment

list_features = ['Temperature', 'Fuel_Price', 'CPI' ,'Unemployment']


for feature in list_features:
    
    # calculating the mean and std for each feature
    # IMPORTANT NOTE the calculation is based on the ORIGINAL DATAFRAME
    # otherwise each time the we filter a feature the std and mean value will change accrodingly 
    # consequently falsifying the results

    std = df[feature].std()      #  standard deviation
    mean = df[feature].mean()    #  mean
    min_val = mean - 3 * std     # always the multiplication is done first before the minus
    max_val = mean + 3 * std      # calculate the upper and lower thresholds


        #======= Excluding the outliers =========#

    mask_outliers = (df2[feature] > min_val) & (df2[feature] < max_val)
    df2 = df2.loc[mask_outliers,:]

        #======= Just Counting the number of affected rows =========#
    
    # this is just to observe the number rows actually resulting from the original datafarme
    mask_outliers = (df[feature] > min_val) & (df[feature] < max_val)
    number_of_rows = df.loc[mask_outliers,:].shape[0]

    print(feature)
    print(f' the {feature} std is {std}')
    print(f' the {feature} mean is {mean}')
    print(f' the {feature} mininmum value is {min_val}')
    print(f' the {feature} maximum value is {max_val}')
    print(f' the number of rows within the threshold are {number_of_rows} out of {df.shape[0]}')
    print(f' The current dataframe number of rows is {df2.shape[0]}')
    print()









Temperature
 the Temperature std is 18.514431869910695
 the Temperature mean is 60.85396694214876
 the Temperature mininmum value is 5.310671332416675
 the Temperature maximum value is 116.39726255188084
 the number of rows within the threshold are 121 out of 136
 The current dataframe number of rows is 136

Fuel_Price
 the Fuel_Price std is 0.47954023536384177
 the Fuel_Price mean is 3.316991935483871
 the Fuel_Price mininmum value is 1.8783712293923458
 the Fuel_Price maximum value is 4.7556126415753965
 the number of rows within the threshold are 124 out of 136
 The current dataframe number of rows is 136

CPI
 the CPI std is 40.24310470097993
 the CPI mean is 178.09114404399998
 the CPI mininmum value is 57.36182994106019
 the CPI maximum value is 298.82045814693976
 the number of rows within the threshold are 125 out of 136
 The current dataframe number of rows is 136

Unemployment
 the Unemployment std is 1.6194283750203007
 the Unemployment mean is 7.665581967213115
 the Unemplo

In [148]:
df.describe()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,136.000000,118,1.360000e+02,125.000000,136.000000,136.000000,136.000000,136.000000
mean,10.014706,2011-04-23 04:28:28.474576384,1.249536e+06,0.072000,60.853967,3.316992,178.091144,7.665582
min,1.000000,2010-02-05 00:00:00,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,2010-07-30 00:00:00,6.050757e+05,0.000000,48.072500,2.853750,132.464884,6.833000
50%,10.000000,2011-04-29 00:00:00,1.261424e+06,0.000000,60.853967,3.316992,189.613975,7.661791
75%,15.250000,2011-12-31 18:00:00,1.806386e+06,0.000000,75.175000,3.689750,214.756314,8.100750
max,20.000000,2012-10-19 00:00:00,2.771397e+06,1.000000,91.650000,4.193000,226.968844,14.313000
std,6.124614,NaN,6.474630e+05,0.259528,17.455574,0.457731,38.568739,1.533160


In [149]:
df2.describe()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,131.000000,113,1.310000e+02,120.000000,131.000000,131.000000,131.000000,131.000000
mean,9.938931,2011-04-24 21:52:33.982300928,1.257990e+06,0.066667,60.453783,3.304198,180.000711,7.427871
min,1.000000,2010-02-05 00:00:00,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,2010-07-30 00:00:00,5.842439e+05,0.000000,47.085000,2.851500,133.361048,6.833000
50%,9.000000,2011-04-22 00:00:00,1.366396e+06,0.000000,60.853967,3.316992,192.826069,7.567000
75%,16.000000,2012-01-13 00:00:00,1.809576e+06,0.000000,75.045000,3.683000,214.781675,8.059000
max,20.000000,2012-10-19 00:00:00,2.771397e+06,1.000000,91.650000,4.170000,226.968844,9.524000
std,6.228663,NaN,6.577463e+05,0.250490,17.444613,0.452979,38.009862,0.942684


Notice how the number of rows the new data frame are unified and the mean and std for each column did not affect each other while masking and we have no 'na's

In [124]:
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,2011-02-18,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,2011-03-25,1807545.43,0.0,42.38,3.435,128.616064,7.470
3,11.0,NaT,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,2010-05-28,1644470.66,0.0,78.89,2.759,212.412888,7.092
5,4.0,2010-05-28,1857533.70,0.0,NaN,2.756,126.160226,7.896


In [125]:
df2.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,2011-02-18,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,2011-03-25,1807545.43,0.0,42.38,3.435,128.616064,7.470
4,6.0,2010-05-28,1644470.66,0.0,78.89,2.759,212.412888,7.092
6,15.0,2011-06-03,695396.19,0.0,69.80,4.069,134.855161,7.658
7,20.0,2012-02-03,2203523.20,0.0,39.93,3.617,213.023622,6.961


## C) ML Preprocessing & Pipelines

In [65]:
df2.dtypes

Store                  float64
Date            datetime64[ns]
Weekly_Sales           float64
Holiday_Flag           float64
Temperature            float64
Fuel_Price             float64
CPI                    float64
Unemployment           float64
dtype: object

In [74]:
# cancelled no need
#df2['Holiday_Flag'] = df2['Holiday_Flag'].apply(lambda x: 'Y' if x == 1 else 'N')

In [84]:
# it is preferred to change the column to string since it is a categorical feature

df2['Store'] = df2['Store'].astype(str)

### C.1) splitting the features from the target

In [119]:
final_list = ['Store','Holiday_Flag','Temperature','Fuel_Price','CPI', 'Unemployment']
x = df2.loc[:,final_list]
y = df2['Weekly_Sales']

print('The features dataset:')
print(x.head())
print()
print('The label series:')
print(y.head())


  Store Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment
0   6.0            N    59.610000    3.045000  214.777523         6.858
1  13.0            N    42.380000    3.435000  128.616064         7.470
3  11.0            N    84.570000    3.316992  214.556497         7.346
4   6.0            N    78.890000    2.759000  212.412888         7.092
5   4.0            N    60.853967    2.756000  126.160226         7.896

0    1572117.54
1    1807545.43
3    1244390.03
4    1644470.66
5    1857533.70
Name: Weekly_Sales, dtype: float64


### Categorizing the feature types and splitting to train and test 

In [120]:
numeric_features = [ 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment']
categorical_features = ['Holiday_Flag','Store']


X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=12)



  Store Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment
0   6.0            N    59.610000    3.045000  214.777523         6.858
1  13.0            N    42.380000    3.435000  128.616064         7.470
3  11.0            N    84.570000    3.316992  214.556497         7.346
4   6.0            N    78.890000    2.759000  212.412888         7.092
5   4.0            N    60.853967    2.756000  126.160226         7.896

0    1572117.54
1    1807545.43
3    1244390.03
4    1644470.66
5    1857533.70
Name: Weekly_Sales, dtype: float64


In [122]:
# Create pipeline for numeric features

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')), # in our case this will be used in the Unmployment column
    ('scaler', StandardScaler())
])



# Create pipeline for categorical features

# this is for the holidays flag column, we must consider it as a categorila feature and not a numeric one

categorical_transformer = Pipeline(
    steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # missing values will be replaced by most frequent value
    ('encoder', OneHotEncoder(drop='first')) # first column will be dropped to avoid creating correlations between features
    ])

In [123]:
# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])



In [124]:
# Preprocessings on train set
print("Performing preprocessings on train set...")
print(X_train.head())
print()

X_train = preprocessor.fit_transform(X_train)

print('...train set transformed and processed.')
print()

print(X_train[0:5]) 
print()


Performing preprocessings on train set...
   Store Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment
89   2.0            N        76.42       3.786  215.154482         7.931
63   5.0            N        69.17       3.594  224.019287         5.422
50  13.0            N        61.11       3.788  130.959226         5.965
10   8.0            N        82.92       3.554  219.070197         6.425
11  18.0            N        52.02       2.878  132.763355         9.331

...Done.
  (0, 0)	0.8871993301852785
  (0, 1)	1.0319553926991747
  (0, 2)	0.9418105339322878
  (0, 3)	0.5128730927917755
  (0, 14)	1.0
  (1, 0)	0.46159700716036506
  (1, 1)	0.6192647064278891
  (1, 2)	1.170524013254484
  (1, 3)	-2.057452836592422
  (1, 18)	1.0
  (2, 0)	-0.011555368505952764
  (2, 1)	1.0362542540145
  (2, 2)	-1.2304414012702571
  (2, 3)	-1.5011806246930135
  (2, 7)	1.0
  (3, 0)	1.2687738266903734
  (3, 1)	0.5332874801213713
  (3, 2)	1.0428366366041597
  (3, 3)	-1.0299371302662572
  (3, 21)	1.0
  (4

Notice the X_train sample, why is it like this and not the usual shape that we know.
this is because it has become a sparce dense matrix with plenty of zero this is due to the oneHotEncoder implementted on the stores column which are too many, 20 stores 

In [97]:
print("And now preprocessings the test set...")

print(X_test.head()) 

X_test = preprocessor.transform(X_test) # DON'T USE FIT otherwise data leakage


print('...Done.')
print(X_test[0:5,:]) 
print()

Performing preprocessings on test set...
    Store Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment
128  13.0            N        75.95    3.606000  129.201581         6.877
79   20.0            N        69.37    3.316992  178.091144         7.527
82   11.0            N        52.77    3.510000  223.917015         6.833
73    1.0            N        85.22    2.619000  211.567306         7.787
21    5.0            N        77.38    3.899000  216.534361         6.489
...Done.
  (0, 0)	0.8596085588995256
  (0, 1)	0.6450578743198445
  (0, 2)	-1.2757889463737535
  (0, 3)	-0.566889174873008
  (0, 7)	1.0
  (1, 0)	0.4733377608989835
  (1, 1)	0.023855080137047277
  (1, 2)	-0.014429894292565894
  (1, 3)	0.09899837159958091
  (1, 15)	1.0
  (2, 0)	-0.5011447994063358
  (2, 1)	0.4387125311842017
  (2, 2)	1.1678853782598762
  (2, 3)	-0.6119646395573097
  (2, 6)	1.0
  (3, 0)	1.4037924946844837
  (3, 1)	-1.4764301847934802
  (3, 2)	0.8492607675387589
  (3, 3)	0.36535339018861773
  (4, 0

## Model training

In [98]:
model_linear = LinearRegression()


In [99]:
model_linear.fit(X_train,y_train)

LinearRegression()

In [100]:
print("Accuracy on training set : ", model_linear.score(X_train, y_train)) # Here, the features must be passed first, and then the true label
print("Accuracy on test set : ", model_linear.score(X_test, y_test))

Accuracy on training set :  0.9662213305521965
Accuracy on test set :  0.9201952696277663


The results are wonderful, 
1. No huge difference between the training and the test accuracy results "only 4%".
2. In addition both values are high. 
3. No indication of huge overfitting since no significant difference between the training and test accuracy
The overfitting case occurs when there's an excellent training score and bad test score. meaning that it only learnt by heart the training set and only knows the training set, if any other data is passed it would fail.

this is not our case, However, let's check if we coul dimprove it more sing either engineering features or the regressors Ridge and Lasso.

BUt before that let's investigate the co-efficients

In [101]:
Y_train_pred = model_linear.predict(X_train)
Y_test_pred = model_linear.predict(X_test)

print("R2 score on training set : ", r2_score(y_train, Y_train_pred))
print("R2 score on test set : ", r2_score(y_test, Y_test_pred))


R2 score on training set :  0.9662213305521965
R2 score on test set :  0.9201952696277663


## model Co efficients:

In [102]:
model_linear.coef_

array([  -18874.90901376,   -50994.70371245,   198924.10643282,
         -37369.19317447,    26501.8002835 ,   766219.67556464,
         -93047.08504421,   874180.4905529 ,   652393.64515598,
        -460915.04447173,  -978467.25327016,  -287055.48790376,
          15204.34606234,   306329.61235054,   297861.19476641,
         442439.40265071, -1176913.42309146,  1019903.82574477,
       -1314669.00751744,   -36251.36942406,  -929885.53593959,
        -747414.87991149, -1143052.09532149])

In [55]:
column_names = []
for name, pipeline, features_list in preprocessor.transformers_: # loop over pipelines
    if name == 'num': # if pipeline is for numeric variables
        features = features_list # just get the names of columns to which it has been applied
    else: # if pipeline is for categorical variables
        features = pipeline.named_steps['encoder'].get_feature_names_out() # get output columns names from OneHotEncoder
    column_names.extend(features) # concatenate features names
        
print("Names of columns corresponding to each coefficient: ", column_names)

Names of columns corresponding to each coefficient:  ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'x0_1.0', 'x1_2.0', 'x1_3.0', 'x1_4.0', 'x1_5.0', 'x1_6.0', 'x1_7.0', 'x1_8.0', 'x1_9.0', 'x1_10.0', 'x1_11.0', 'x1_13.0', 'x1_14.0', 'x1_15.0', 'x1_16.0', 'x1_17.0', 'x1_18.0', 'x1_19.0', 'x1_20.0']


In [57]:
# Create a pandas DataFrame
coefs = pd.DataFrame(index = column_names, data = model.coef_.transpose(), columns=["coefficients"])
coefs

,coefficients
Temperature,-1.887491e+04
Fuel_Price,-5.099470e+04
CPI,1.989241e+05
Unemployment,-3.736919e+04
x0_1.0,2.650180e+04
x1_2.0,2.978612e+05
x1_3.0,-1.176913e+06
x1_4.0,1.019904e+06
x1_5.0,-1.314669e+06
x1_6.0,-3.625137e+04


# Create a pandas DataFrame
coefs = pd.DataFrame(index = column_names, data = regressor.coef_.transpose(), columns=["coefficients"])
coefs

In [127]:
# Assuming you have trained your model and obtained the coefficients

# Step 1: Get the coefficients
coefficients = model.coef_

# Step 2: Get the feature names
feature_names = []

# Assuming you used the ColumnTransformer to preprocess your data
for transformer_in_columns in preprocessor.transformers_:
    transformer_name, transformer, columns = transformer_in_columns
    if transformer_name == 'num':
        feature_names.extend(columns)
    elif transformer_name == 'cat':
        one_hot_encoder = transformer.named_steps['encoder']
        feature_names.extend(one_hot_encoder.get_feature_names_out(columns))

# Step 3: Match coefficients with feature names
coefficients_with_features = dict(zip(feature_names, coefficients))

# Print the coefficients with their corresponding feature names
for feature, coefficient in coefficients_with_features.items():
    print(f"{feature}: {coefficient}")

coefficients_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients})

# Step 4: Sort the DataFrame in descending order based on coefficient magnitude
coefficients_df = coefficients_df.sort_values(by='Coefficient', ascending=False)


# Print the sorted DataFrame
print(coefficients_df)

Temperature: -18874.909013755027
Fuel_Price: -50994.70371244922
CPI: 198924.10643281622
Unemployment: -37369.19317446862
Holiday_Flag_Y: 26501.800283496792
Store_10.0: 766219.675564637
Store_11.0: -93047.08504421315
Store_13.0: 874180.490552895
Store_14.0: 652393.6451559784
Store_15.0: -460915.0444717274
Store_16.0: -978467.2532701641
Store_17.0: -287055.48790376156
Store_18.0: 15204.346062341638
Store_19.0: 306329.6123505404
Store_2.0: 297861.19476641
Store_20.0: 442439.4026507138
Store_3.0: -1176913.4230914586
Store_4.0: 1019903.8257447692
Store_5.0: -1314669.0075174375
Store_6.0: -36251.36942406133
Store_7.0: -929885.5359395852
Store_8.0: -747414.8799114854
Store_9.0: -1143052.095321491
           Feature   Coefficient
17       Store_4.0  1.019904e+06
7       Store_13.0  8.741805e+05
5       Store_10.0  7.662197e+05
8       Store_14.0  6.523936e+05
15      Store_20.0  4.424394e+05
13      Store_19.0  3.063296e+05
14       Store_2.0  2.978612e+05
2              CPI  1.989241e+05
4   

Regressor

In [132]:
# Perform 3-fold cross-validation to evaluate the generalized R2 score obtained with a Ridge model
# usually the values chosen for k are 3, 5 or 10

print("3-fold cross-validation...")
model_regressor = Ridge(0)
scores = cross_val_score(model_regressor, X_train, y_train, cv=3)
print('The cross-validated R2-score is : ', scores.mean())
print('The standard deviation is : ', scores.std())

model_regressor = Ridge(1)
scores = cross_val_score(model_regressor, X_train, y_train, cv=3)
print('The cross-validated R2-score is : ', scores.mean())
print('The standard deviation is : ', scores.std())


model_regressor = Ridge(0.3)
scores = cross_val_score(model_regressor, X_train, y_train, cv=3)
print('The cross-validated R2-score is : ', scores.mean())
print('The standard deviation is : ', scores.std())

3-fold cross-validation...
The cross-validated R2-score is :  0.9267035624854393
The standard deviation is :  0.01619758906143055
The cross-validated R2-score is :  0.8476556737731308
The standard deviation is :  0.02657474230307866
The cross-validated R2-score is :  0.9156525066483695
The standard deviation is :  0.0064732957495451495


In [129]:
# Perform grid search
print("Grid search...")
model_regressor = Ridge()
# Grid of values to be tested
params = {
    'alpha': [0.0, 0.1, 0.5, 1.0, 2] # 0 corresponds to no regularization
}
gridsearch = GridSearchCV(model_regressor, param_grid = params, cv = 3) # cv : the number of folds to be used for CV

print("Trying various hyperparameters and searching the optimum results.")

gridsearch.fit(X_train, y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch.best_params_)
print("Best R2 score : ", gridsearch.best_score_)

Grid search...
...Done.
Best hyperparameters :  {'alpha': 0.1}
Best R2 score :  0.9293317637533356


In [142]:
# Perform grid search
print("Grid search...")
model_regressor_lasso = Lasso()
# Grid of values to be tested
params = {
    'alpha': [1, 2, 3, 5, 10, 20, 200] 
}
gridsearch = GridSearchCV(model_regressor_lasso, param_grid = params, cv = 3) # cv : the number of folds to be used for CV

print("Trying various hyperparameters and searching the optimum results.")

gridsearch.fit(X_train, y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch.best_params_)
print("Best R2 score : ", gridsearch.best_score_)

Grid search...
Trying various hyperparameters and searching the optimum results.
...Done.
Best hyperparameters :  {'alpha': 200}
Best R2 score :  0.9292495504331088


Preleminary Analysis

In [42]:
import plotly.express as px

In [62]:

list_features = ['Date','Holiday_Flag','Temperature','Fuel_Price','CPI', 'Unemployment']
for f in list_features:
    fig = px.scatter(dataset, x=dataset[f], y="Weekly_Sales")
    fig.show()

In [45]:
dataset.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,2011-02-18,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,2011-03-25,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,2012-07-27,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaT,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,2010-05-28,1644470.66,0.0,78.89,2.759,212.412888,7.092


In [50]:
import plotly.graph_objects as go


In [67]:
fig = px.scatter_matrix(dataset)
fig.update_layout(
        title = go.layout.Title(text = "Bivariate analysis", x = 0.5), showlegend = False, 
            autosize=False, height=1100, width = 1100)
fig.show()

### Investigate correlation

In [174]:
corr_matrix = dataset.corr().round(2)

In [171]:
import plotly.figure_factory as ff

In [175]:
fig = ff.create_annotated_heatmap(corr_matrix.values, x=corr_matrix.columns.tolist(), y = corr_matrix.index.tolist())
fig.show()

the correlation matrix shows two main observations, the first is that a considerable correlation exist between the weekly sales and the CPI, un-employment, Temperature, and the store branch of wallmart. actually thinking about it for a second, in the end of day what concerns us is the whole weekly sale regardless of the store branch, consequently it would be fine to take it off, unless we need to analyse the contribution and effect of each store.

##### the second observation is that there is no ctrong correlation between the features themselves , i.e. there is not stong inter-correlation between the columns that might affect the regressionlinear prediction